![Alt text](http://www.ucm.es/logo/ucm.png "a title")

<div align="center"> 
<font size=5> Máster en Nuevas Tecnologías Electrónicas y Fotónicas </font>
</div>
    
<div align="center"> 
<font size=4> Óptica Digital, curso 2022-2023 </font>
</div>

    
<div align="center"> 
<font size=4> Ejercicio 1 - Propagación de la luz </font>
</div>

<style>
.alert-green {
  color: rgb(60,118,61) !important;
}
</style>


- **Fecha**: ***
        
- **Alumno**: ****   


# Introducción

1. En el marco XZ, cree un haz gaussiano con longitud de onda $\lambda = 2\mu m$, y observe cómo varía su propagación (amplitud y fase) a medida que cambia la anchura de su cintura, para w0=25 $\mu m$ y w0=2 $\mu m$. Para la propagación utilice la función WPM.

1. En el marco XY, cree una máscara que esté formada por dos aberturas: 1 círculo de 50 micras de diámetro y ubicado en la posición (x,y)=(-100 um,0) y un cuadrado de 100 micras dde lado y ubicado en la posición (x,y)=(100 um, 0). Se ilumina por una onda armónica plana de  $\lambda = 0.6328\mu m$ y se obsera el campo a una distancia de 2 mm. Para la propagación utilice la función RS.

1. En el marco XY, vamos a utilizar la máscara square_circle que es una función entre un cuadrado y un círculo, según el parámetro s entre 0 y 1, donde s=0 es un círculo y s=1 es un cuadrado. Queremos ver la figura de difracción en campo lejano. Cree una máscara de 100 $\mu m$ de tamaño (R1=100*um, R2=100*um) y utilice una lente de focal de 2 mm y 200 um de radio para observar la figura de difracción en el plano focal de la lente. Determine la intensidad luminosa para s=0.75 y s=0.95. Para la propagación utilice la función RS.

    Tenga como referencia para los resultados el artículo M. Fernandez Guasti and M. De La Cruz Heredia, “Diffraction pattern of a circle/square aperture,” J. Mod. Opt., vol. 40, no. 6, pp. 1073–1080, 1993.
    
    Para preguntar por una función determinada, en jupyter, puede realizar lo siguiente:
    't0.square_circle?' donde t0 es una máscara definida previamente.


Proponga un valor para los parámetros que no vengan en el enunciado. En todos los casos, los resultados son gráficas de intensidad y fase luminosa.

## Resolución 1


**Conclusiones**: 

## Resolución 2

**Conclusiones**: 

## Resolución 3

**Conclusiones**: 